In [ ]:
!pip install --upgrade --quiet langchain langchain-community google-ai-generativelanguage==0.6.15 langchain-google-genai langchain-experimental neo4j langchain-neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 21.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.13.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
import os
from langchain_neo4j import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from google.colab import userdata


In [ ]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['NEO4J_URI'] = userdata.get('NEO4J_URI')
os.environ['NEO4J_USERNAME']= userdata.get('NEO4J_USERNAME')
os.environ['NEO4J_PASSWORD'] = userdata.get('NEO4J_PASSWORD')

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
llm_transformer = LLMGraphTransformer(llm = llm)

In [ ]:
graph = Neo4jGraph()

/tmp/ipython-input-267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [ ]:
text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""

In [ ]:
documents = [Document(page_content = text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Marie Curie', type='Person', properties={}), Node(id='Pierre Curie', type='Person', properties={}), Node(id='University Of Paris', type='Organization', properties={}), Node(id='Nobel Prize', type='Award', properties={})], relationships=[Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='Pierre Curie', type='Person', properties={}), type='SPOUSE', properties={}), Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='Nobel Prize', type='Award', properties={}), type='AWARDED', properties={}), Relationship(source=Node(id='Pierre Curie', type='Person', properties={}), target=Node(id='Nobel Prize', type='Award', properties={}), type='AWARDED', properties={}), Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='University Of Paris', type='Organization', properties={}), type='PROFESSOR', properties={})], source=Document(metadata={}, page_content='\nMar

In [ ]:
for node in graph_documents[0].nodes:
  print(f'id = {node.id}  type = {node.type}')

id = Marie Curie  type = Person
id = Pierre Curie  type = Person
id = University Of Paris  type = Organization
id = Nobel Prize  type = Award


In [ ]:
for relationship in graph_documents[0].relationships:
  print(relationship)

source=Node(id='Marie Curie', type='Person', properties={}) target=Node(id='Pierre Curie', type='Person', properties={}) type='SPOUSE' properties={}
source=Node(id='Marie Curie', type='Person', properties={}) target=Node(id='Nobel Prize', type='Award', properties={}) type='AWARDED' properties={}
source=Node(id='Pierre Curie', type='Person', properties={}) target=Node(id='Nobel Prize', type='Award', properties={}) type='AWARDED' properties={}
source=Node(id='Marie Curie', type='Person', properties={}) target=Node(id='University Of Paris', type='Organization', properties={}) type='PROFESSOR' properties={}


In [ ]:
graph.add_graph_documents(graph_documents)


In [ ]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {id: STRING}
Organization {id: STRING}
Award {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:SPOUSE]->(:Person)
(:Person)-[:AWARDED]->(:Award)
(:Person)-[:PROFESSOR]->(:Organization)


In [ ]:
# chain
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph = graph, llm=llm, verbose=True, allow_dangerous_requests=True)

In [ ]:
response = chain.invoke({'query': 'Marie Curie was professor at which university'})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {id: "Marie Curie"})-[:PROFESSOR]->(o:Organization)
RETURN o.id

Full Context:
[{'o.id': 'University Of Paris'}]

> Finished chain.


{'query': 'Marie Curie was professor at which university',
 'result': 'Marie Curie was a professor at the University Of Paris.'}